In [63]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sc
import math
import datetime
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn import tree
from sklearn import metrics

In [64]:
validation_df = pd.read_csv("flights_train.csv")
validation_y = validation_df['ARR_DELAY']

In [65]:
day_of_year = [datetime.datetime.strptime(date_str, '%Y-%m-%d').timetuple().tm_yday for date_str in validation_df['FL_DATE'] ]
month = [datetime.datetime.strptime(date_str, '%Y-%m-%d').timetuple().tm_mon for date_str in validation_df['FL_DATE'] ]
validation_x = validation_df.drop(['UID','FL_NUM','AIRLINE_ID','ORIGIN_CITY_NAME','DISTANCE', 'FL_DATE', 'DEST_CITY_NAME','ORIGIN_STATE_ABR','DEST_STATE_ABR', 'FIRST_DEP_TIME'], 
                 axis=1)  
validation_x['month'] = month
validation_x['day_of_year'] = day_of_year

validation_x = validation_x.drop(['UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'ORIGIN_CITY_MARKET_ID', 'DEST_CITY_MARKET_ID'], axis=1)

In [66]:
carrier_ohe = pd.get_dummies(validation_df['UNIQUE_CARRIER'])
carrier_ohe = carrier_ohe.astype('int64')
origin_ohe = pd.get_dummies(validation_df['ORIGIN'])
origin_ohe = origin_ohe.astype('int64')
dest_ohe = pd.get_dummies(validation_df['DEST'])
dest_ohe.rename(columns=lambda x: x+'_DEST', inplace=True)
dest_ohe = dest_ohe.astype('int64')
nn_validation_x = pd.concat([validation_x, carrier_ohe, origin_ohe, dest_ohe], axis=1) 

In [67]:
nn_validation_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4911 entries, 0 to 4910
Columns: 488 entries, DAY_OF_WEEK to YUM_DEST
dtypes: int64(488)
memory usage: 18.3 MB


In [68]:
validation_x[0:2]

,DAY_OF_WEEK,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE_GROUP,ARR_DELAY,month,day_of_year
0,2,1020,11,8,373,11,-11,1,3
1,6,1220,13,9,183,5,1,1,28


In [69]:
y_mean = validation_df['ARR_DELAY'].mean()
y_std = validation_df['ARR_DELAY'].std()
y_max = y_mean + (2 * y_std)
y_min = y_mean - (2 * y_std)

training_x = validation_x[validation_x['ARR_DELAY'] < y_max]
nn_training_x = nn_validation_x[validation_x['ARR_DELAY'] < y_max]
training_y = training_x['ARR_DELAY']

In [70]:

training_x = training_x.drop(['ARR_DELAY'], axis=1)

nn_training_x = nn_training_x.drop(['ARR_DELAY'], axis=1)

In [74]:
kbest = SelectKBest(mutual_info_regression, k=4).fit(training_x, training_y)
kbest.get_support()

/Users/lauraedelson/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


array([False,  True,  True, False,  True, False, False,  True], dtype=bool)

In [77]:
clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(training_x, training_y)


In [78]:
with open("tree.dot", 'w') as f:
    f = tree.export_graphviz(clf, out_file=f)
